In [199]:
import pandas as pd
from tqdm import tqdm

In [200]:
# TODO house_interaction에 클러스터 결과를 merge해줍니다. 이때, 클러스터 그룹이 -1이거나 house_interaction에만 포함된 item들은 cluster_id.max() + 1을 더해줍니다.
# TODO model을 학습해서 학습한 결과를 backend에 반영합니다.
# TODO cluster-major-item list를 생성합니다.

In [201]:
item = pd.read_csv("/opt/ml/input/final/EDA/clustered_item_09305.csv")
train = pd.read_csv("/opt/ml/input/data/hi_interaction.csv")

In [202]:
item.head(1)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
0,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈59.0 × 37.0 × 90.0 (cm)출처www.casa.co.kr,66808,[],[],-1


In [203]:
train.head(1)

,house,item
0,318,55


In [204]:
item.cls_id.nunique()

8740

In [205]:
item.item.nunique()

88634

In [206]:
len(train)

412784

In [207]:
# item에 포함되지 않은 hi_interaction item을 제거하기
train = train[train.item.isin(item.item.unique())]

In [208]:
len(train)

257305

In [209]:
item.cls_id.max()

9190

In [210]:
len(item[item.cls_id == -1])

63595

In [211]:
dummy = list()
c = item.cls_id.max()

for i in range(len(item[item.cls_id == -1])):
    dummy.append(i + c + 1)

In [212]:
item.loc[item.cls_id == -1, "cls_id"] = dummy

In [213]:
item.head(3)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
0,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈59.0 × 37.0 × 90.0 (cm)출처www.casa.co.kr,66808,[],[],9191
1,4,데코·식물|시계|벽시계,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,118400,우드피아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈22.0 × 22.0 × 1.5 (cm)출처www.11st.co.kr,118400,[],[],9192
2,8,가구|거실장·TV장|일반거실장,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,131836,DOICH(도이치),NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈165.0 × 40.0 × 37.5 (cm)출처m.doich.co.kr,131836,[],[],9193


In [214]:
item2cls = {item_id:cls_id for item_id, cls_id in zip(item.item, item.cls_id)}

In [215]:
train["cluster_id"] = train.item.map(item2cls)

In [216]:
train

,house,item,cluster_id
0,318,55,0
1,318,1490,17
4,318,2256,10067
5,318,2257,10068
6,318,2258,10069
...,...,...,...
412761,135273,1169670,65113
412764,135273,1451760,69894
412765,135273,1467566,70090
412767,135273,1709921,72269


In [217]:
train.to_csv("clustered_train.csv", index=False)

In [218]:
predict_price_list = []

import re
p = re.compile('.+(?=사)')
for i in item.predict_price.index:
    m = p.findall(str(item.predict_price.iloc[i]))
    if len(m) == 0:
        m.append("예상가정보없음")
    if m[0][3:] == "정보없음":
        predict_price_list.append("정보없음")
    else:
        predict_price_list.append(int("".join((m[0][3:]).split(","))))

In [219]:
item.predict_price = predict_price_list

In [220]:
item.loc[(item.predict_price != "정보없음"), "price"] = item[(item.predict_price != "정보없음")].predict_price

In [221]:
item

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
0,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,66808,[],[],9191
1,4,데코·식물|시계|벽시계,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,118400,우드피아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,118400,[],[],9192
2,8,가구|거실장·TV장|일반거실장,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,131836,DOICH(도이치),NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,131836,[],[],9193
3,9,가구|의자|스툴·벤치,별점 5.0점,(2)개 리뷰,NaN,204593,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,204593,[],[],9194
4,10,가구|화장대·콘솔|일반화장대,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,221527,아노말리,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,221527,[],[],9195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88629,1820832,수납·정리|화장대·테이블정리|소품정리·거치대,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,143000,서라운드 플로 스탠드 ufs 20 ii,BOSE,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,143000,서라운드 플로어 스탠드 UFS-20 II,[],[],72781
88630,1821165,조명|LED등|LED방등,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,arwen ceiling light,이라이트,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,[해외]Arwen Ceiling Light 450S/550S,[],[],72782
88631,1821312,가전·디지털|게임기·드론|게임기,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,360000,nintendo switch 포트 나이트 스페셜 세트,닌텐도,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,360000,Nintendo Switch 포트나이트 스페셜 세트,[],[],72783
88632,1822506,데코·식물|디퓨저·캔들|홈퍼퓸·사쉐,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,eau triple saint joseph charpentier,불리1803,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,[해외] EAU TRIPLE SAINT JOSEPH CHARPENTIER,[],[],72784


In [222]:
import re
def f(x):
    pattern = re.compile(r"별점\s+(\d+\.\d+)점")
    try:
        match = pattern.search(x)
        if match:
            score = float(match.group(1))
        else:
            score = None
        return score
    except:
        return 0.0

In [223]:
item.rating = item.rating.apply(lambda x: f(x))

In [224]:
item.review = item.review.fillna("리뷰 쓰기첫 리뷰 두 배 적립")
review_list = []

import re
p = re.compile('[(][0-9]*[)]')
for idx, review in enumerate(item.review):
    m = p.findall(review)
    if len(m) == 0:
        m.append("(0)")
    review_list.append(m[0][1:-1])

item.review = review_list

In [225]:
item.review = item.review.astype(int)

In [226]:
item.rating = item.rating.astype(float)

In [227]:
item.head(5)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
0,1,가구|서랍·수납장|수납장,4.2,67,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,66808,[],[],9191
1,4,데코·식물|시계|벽시계,0.0,0,NaN,118400,우드피아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,118400,[],[],9192
2,8,가구|거실장·TV장|일반거실장,0.0,0,NaN,131836,DOICH(도이치),NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,131836,[],[],9193
3,9,가구|의자|스툴·벤치,5.0,2,NaN,204593,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,204593,[],[],9194
4,10,가구|화장대·콘솔|일반화장대,0.0,0,NaN,221527,아노말리,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,221527,[],[],9195


In [228]:
item = item.sort_values(["cls_id", "review", "rating"], ascending=[True, False, False])

In [229]:
item

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
44601,456458,가구|선반|스탠드선반,4.7,728,NaN,lerberg 레르 베리 선반 유닛 단,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,정보없음,LERBERG 레르베리 선반유닛 2단,"[['lerberg 레르 베리 선반 유닛 단', 'lerberg 레르 베리 선반 유...","[1559246, 456462, 980622, 456449, 456455, 4564...",0
44604,456462,가구|선반|스탠드선반,4.7,425,NaN,lerberg 레르 베리 코너 선반 유닛,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,정보없음,LERBERG 레르베리 코너선반유닛,"[['lerberg 레르 베리 코너 선반 유닛', 'lerberg 레르 베리 트레슬...","[980622, 188569, 456449, 456455, 456458, 98062...",0
17,55,가구|선반|스탠드선반,4.7,409,NaN,lerberg 철제 선반 l,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,LERBERG 철제선반 L 4colors,"[['lerberg 철제 선반 l', 'lerberg 철제 선반 s', 769, 0...","[769, 236215, 769, 7390, 133699, 133700, 15378...",0
44599,456449,가구|선반|스탠드선반,4.8,385,NaN,lerberg 레르 베리 선반 유닛 s,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,정보없음,LERBERG 레르베리 선반유닛 S,"[['lerberg 레르 베리 선반 유닛 s', 'lerberg 레르 베리 선반 유...","[456458, 1559246, 456462, 980622, 456455, 4564...",0
19329,138899,가구|선반|스탠드선반,4.7,354,NaN,lerberg 레르 베리 단 단슬림 와이드 철제 선반,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,LERBERG 레르베리 2단/4단슬림/4단와이드 철제선반 3colors,"[['lerberg 레르 베리 단 단슬림 와이드 철제 선반', 'lerberg 레르...","[338761, 338775, 338755, 338755, 338761, 338775]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88629,1820832,수납·정리|화장대·테이블정리|소품정리·거치대,0.0,0,143000,서라운드 플로 스탠드 ufs 20 ii,BOSE,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,143000,서라운드 플로어 스탠드 UFS-20 II,[],[],72781
88630,1821165,조명|LED등|LED방등,0.0,0,NaN,arwen ceiling light,이라이트,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,[해외]Arwen Ceiling Light 450S/550S,[],[],72782
88631,1821312,가전·디지털|게임기·드론|게임기,0.0,0,360000,nintendo switch 포트 나이트 스페셜 세트,닌텐도,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,360000,Nintendo Switch 포트나이트 스페셜 세트,[],[],72783
88632,1822506,데코·식물|디퓨저·캔들|홈퍼퓸·사쉐,0.0,0,NaN,eau triple saint joseph charpentier,불리1803,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,[해외] EAU TRIPLE SAINT JOSEPH CHARPENTIER,[],[],72784


In [230]:
item.groupby("cls_id")["item"].apply(lambda x:len(list(x)))

cls_id
0        22
1         3
2         2
3         2
4         2
         ..
72781     1
72782     1
72783     1
72784     1
72785     1
Name: item, Length: 72334, dtype: int64

In [239]:
cls_major_item_id = item.rename(columns={"item":"major_item"}).groupby("cls_id")["major_item"].apply(lambda x:list(x)[0]).reset_index()
cls_major_item_id_list = item.rename(columns={"item":"item_list"}).groupby("cls_id")["item_list"].apply(lambda x:"|".join(list(map(str, list(set(x)))))).reset_index()

In [240]:
cls_major_item_id.merge(cls_major_item_id_list, on="cls_id")

,cls_id,major_item,item_list
0,0,456458,456449|769|456455|456458|456462|980622|138899|...
1,1,1017200,1017200|89|743437
2,2,103,350421|103
3,3,107,107|1436
4,4,108,108|143
...,...,...,...
72329,72781,1820832,1820832
72330,72782,1821165,1821165
72331,72783,1821312,1821312
72332,72784,1822506,1822506


In [242]:
cls_major_item_id.to_csv("cluster_major_item.csv", index=True)